# Customer Segementation for Online Retailing - Data Wrangling

**Abstract:** This project is based on a transnational dataset which contains all the transactions occurring between 1/12/2010 and 9/12/2011 for a UK-based and registered non-store online retail. Through data analysis and modeling,  this project will focus on customer segmentation that will help the retailer make effective marketing decisions to increase its sales and profits.  

## Data Wrangling

### Data Collection

In [1]:
#Load python packages
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
%matplotlib inline

color = sns.color_palette()

In [2]:
df = pd.read_excel('G:\My Drive\Jun\Mechine Learning\Data Science\Capstone Two\Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Description of columns

| Column | Description |
| :- | :- |
| InvoiceNo | Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cacellation. |
| StockCode | Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. |
| Description | Product (item) name. Nominal. |
| Quantity | The quantities of each product (item) per transaction. Numeric. |
| InvoiceDate | Invoice Date and time. Numeric, the day and time when each transaction was generated. |
| UnitPrice | Unit price. Numeric. Product price per unit in sterling. |
| CustomerID | Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. |
| Country | Country name. Nominal, the name of the country where each customer resides. |


###  Data Cleaning

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
# Print the range of values using the aggregate function. 
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


####  Duplicates

In [5]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536641 non-null  object        
 1   StockCode    536641 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     536641 non-null  int64         
 4   InvoiceDate  536641 non-null  datetime64[ns]
 5   UnitPrice    536641 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      536641 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.8+ MB


####  Missing values

In [6]:
nas = pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df), columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
CustomerID,0.251634
Description,0.002709


Two variables have missing values. Let's look at them individually and think about filling methods. 

**Description**: each product has a unique StockCode and description. Therefore it is possible to fill missing discription using StockCode. Bear in mind that in later analysis, only one of StockCode and Description will be used in analysis and modelling, because they are highly correlated. 

In [7]:
df['Description'] = df.groupby(["StockCode"])['Description'].transform(lambda x: x.fillna(x.mode()))

nas = pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df), columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]

,percent
CustomerID,0.251634
Description,0.002709


**CustomerID**: each transaction record involves an InvoiceNo, CustomerID and Country. Therefore we might be able to fill in missing CustomerID using InvoiceID and Country. 

In [8]:
df_Non_Null = df[df['CustomerID'].isnull()==False].copy()
print(df_Non_Null.shape)

df_Null = df[df['CustomerID'].isnull()==True].copy()
print(df_Null.shape)

df_join = df_Non_Null.merge(df_Null, on = 'InvoiceNo', how = 'inner')
df_join.head()

(401604, 8)
(135037, 8)


,InvoiceNo,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID_x,Country_x,StockCode_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,CustomerID_y,Country_y


In [9]:
# since we were not able to find appropriate ways to fill the missing data, we will drop the missing values.
df = df.dropna(subset=['Description', 'CustomerID'])

####  Data Types

In [10]:
# set the data types: all columncs look good except that "CustomerID" should be string.
df['CustomerID'] = df['CustomerID'].astype(int).astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    401604 non-null  object        
 1   StockCode    401604 non-null  object        
 2   Description  401604 non-null  object        
 3   Quantity     401604 non-null  int64         
 4   InvoiceDate  401604 non-null  datetime64[ns]
 5   UnitPrice    401604 non-null  float64       
 6   CustomerID   401604 non-null  object        
 7   Country      401604 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.6+ MB


####  Anomalies and Outliers

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,401604.0,12.183273,250.283037,-80995.0,2.00,5.00,12.00,80995.0
UnitPrice,401604.0,3.474064,69.764035,0.0,1.25,1.95,3.75,38970.0


The distribution of Quantity and UnitPrice is highly skewed. While most purchases are about small Quantity and small UnitPrice, there are a few extra large amount Quantity and UnitPrice.

In [12]:
df[(df.Description.str.len())<15]['Description'].unique()

array(['POSTAGE', 'Discount', 'CHILLI LIGHTS', 'PHOTO CUBE',
       'RETROSPOT LAMP', 'DOORMAT HEARTS', 'WRAP COWBOYS  ',
       'JUMBO BAG OWLS', 'OWL DOORSTOP', 'WICKER STAR ', 'FIRST AID TIN',
       'BUTTON BOX ', 'POLKADOT PEN', 'CARRIAGE', 'RETRO MOD TRAY',
       'Manual', 'BINGO SET', 'CUTE CATS TAPE', 'GLAMOROUS  MUG',
       'LOCAL CAFE MUG', 'Bank Charges', 'SPACE FROG', 'SPACE OWL',
       'KEY FOB , SHED', 'DOGGY RUBBER', 'SKULLS TAPE', 'PARTY BUNTING',
       'PINK DOG BOWL', 'POTTERING MUG', 'SANDALWOOD FAN', 'SOMBRERO ',
       'RAIN PONCHO ', 'MIRROR CORNICE', 'DAISY JOURNAL ',
       'POSY CANDY BAG', 'GOLD WASHBAG', 'WRAP FOLK ART', 'PINK CAT BOWL',
       'FUNKY DIVA PEN', 'RIBBONS PURSE ', 'POMPOM CURTAIN',
       'PINK FLY SWAT', 'BLUE FLY SWAT', 'FROG CANDLE', 'TOYBOX  WRAP ',
       'WRAP, CAROUSEL', 'BUNNY EGG BOX', 'WRAP CAROUSEL',
       'SPOTTY BUNTING', 'LED TEA LIGHTS', 'POPCORN HOLDER',
       'GARAGE KEY FOB', 'PACKING CHARGE', 'CORDIAL JUG',
       'DOT

In [13]:
df[(df.StockCode.str.len())<5]['StockCode'].unique()

array(['POST', 'D', 'C2', 'M', 'PADS', 'DOT', 'CRUK'], dtype=object)

In [14]:
df[(df.StockCode.str.len())>5]['StockCode'].unique()

array(['85123A', '84406B', '84029G', '84029E', '82494L', '85099C',
       '84997B', '84997C', '84519A', '85183B', '85071B', '37444A',
       '37444C', '84971S', '15056BL', '15056N', '35004C', '85049A',
       '85099B', '35004G', '85014B', '85014A', '84970S', '84030E',
       '35004B', '85049E', '17091A', '84509A', '84510A', '84709B',
       '84625C', '84625A', '47570B', '85049C', '85049D', '85049G',
       '84970L', '90199C', '90129F', '90210B', '72802C', '85169B',
       '85099F', '85184C', '35591T', '84032B', '85049H', '72800E',
       '84849B', '90200B', '90059B', '90185C', '90059E', '90059C',
       '90200C', '90200D', '90200A', '16258A', '85231B', '85231G',
       '48173C', '47563A', '84558A', '46000M', '71406C', '84985A',
       '84596E', '84997D', '47599A', '47599B', '85035B', '84968C',
       '72800B', '84563A', '47504H', '17164B', '15044B', '84569B',
       '85114B', '85114C', '85199L', '85199S', '85019A', '85019C',
       '85071A', '85071C', '85135B', '85136A', '85136C', '791

In [15]:
df_cancelled = df[df.Quantity < 0]
df_cancelled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8872 entries, 141 to 541717
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    8872 non-null   object        
 1   StockCode    8872 non-null   object        
 2   Description  8872 non-null   object        
 3   Quantity     8872 non-null   int64         
 4   InvoiceDate  8872 non-null   datetime64[ns]
 5   UnitPrice    8872 non-null   float64       
 6   CustomerID   8872 non-null   object        
 7   Country      8872 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 623.8+ KB


In [16]:
df_cancelled.to_csv(r'G:\My Drive\Jun\Mechine Learning\Data Science\Capstone Two\CancelledOrders.csv')

In [17]:
def remove_cancelled_transactions(df):
    trans_neg = df.Quantity < 0
    return df.loc[~(trans_neg | trans_neg.shift(-1))]

groups = [df.CustomerID, df.StockCode, df.Quantity.abs()]
df1 = df.groupby(groups, as_index=False, group_keys=False).apply(remove_cancelled_transactions)

In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 389872 entries, 148306 to 198740
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    389872 non-null  object        
 1   StockCode    389872 non-null  object        
 2   Description  389872 non-null  object        
 3   Quantity     389872 non-null  int64         
 4   InvoiceDate  389872 non-null  datetime64[ns]
 5   UnitPrice    389872 non-null  float64       
 6   CustomerID   389872 non-null  object        
 7   Country      389872 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 26.8+ MB


In [19]:
df1.to_csv(r'G:\My Drive\Jun\Mechine Learning\Data Science\Capstone Two\CancelledOrdersRemoved.csv')

In [20]:
df1 = pd.read_csv('G:\My Drive\Jun\Mechine Learning\Data Science\Capstone Two\CancelledOrdersRemoved.csv')
df1.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,148306,549222,16008,SMALL FOLDING SCISSOR(POINTED EDGE),24,2011-04-07 10:43:00,0.25,12347,Iceland
1,220583,556201,17021,NAMASTE SWAGAT INCENSE,36,2011-06-09 13:01:00,0.30,12347,Iceland
2,148289,549222,20665,RED RETROSPOT PURSE,6,2011-04-07 10:43:00,2.95,12347,Iceland
3,72249,542237,20719,WOODLAND CHARLOTTE BAG,10,2011-01-26 14:30:00,0.85,12347,Iceland
4,286629,562032,20719,WOODLAND CHARLOTTE BAG,10,2011-08-02 08:48:00,0.85,12347,Iceland


In [21]:
df1[(df1.StockCode.str.len())<5]['StockCode'].unique()

array(['POST', 'M', 'C2', 'PADS', 'DOT'], dtype=object)

In [22]:
df1[(df1.StockCode.str.len())>5]['StockCode'].unique()

array(['47559B', '47567B', '51014C', '84558A', '84559A', '84559B',
       '84625A', '84625C', '84997B', '84997C', '84997D', '85167B',
       '85232D', '47504H', '84078A', '85014A', '85014B', '79066K',
       '79191C', '84086C', '84510A', '72802A', '72802B', '72802C',
       '85040A', '84535B', '84569D', '85184C', '15056BL', '15056P',
       '35598B', '35598D', '35599B', '35599D', '84029E', '84029G',
       '84030E', '85159A', '85159B', '15056N', '15060B', '82613B',
       '82613C', '82613D', '85099B', '85099C', '85099F', '84997A',
       '85185B', '47566B', '47591D', '82001S', '84569A', '84569B',
       '85230B', '16156L', '16156S', '16161U', '16169M', '47590A',
       '47590B', '72807A', '84884A', '18097C', '35912B', '35913B',
       '47574A', '79302M', '85123A', '85180B', '85194L', '85199L',
       '85199S', '17012D', '85034A', '85034C', '85035A', '85035B',
       '85035C', '85036A', '85036B', '85036C', '46775D', '85206A',
       '15058B', '47599A', '84912A', '85049G', '16169E', '354

In [23]:
df1[(df1.StockCode.str.isalpha())>0]['StockCode'].unique()

array(['POST', 'M', 'PADS', 'DOT'], dtype=object)

In [24]:
df2 = df1.drop(df1[(df1.StockCode.str.len())<5].index).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388390 entries, 0 to 388389
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   388390 non-null  int64  
 1   InvoiceNo    388390 non-null  int64  
 2   StockCode    388390 non-null  object 
 3   Description  388390 non-null  object 
 4   Quantity     388390 non-null  int64  
 5   InvoiceDate  388390 non-null  object 
 6   UnitPrice    388390 non-null  float64
 7   CustomerID   388390 non-null  int64  
 8   Country      388390 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 26.7+ MB


In [25]:
df2 = df2.drop(df2[(df2.StockCode.str.len())>6].index).reset_index(drop=True)

In [26]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,388090.0,278441.393684,152607.755537,0.0,148556.50,284835.50,409844.75,541908.0
InvoiceNo,388090.0,560616.141926,13092.078794,536365.0,549242.00,561889.00,572076.00,581587.0
Quantity,388090.0,12.702517,46.753003,1.0,2.00,6.00,12.00,12540.0
UnitPrice,388090.0,2.851258,3.983498,0.0,1.25,1.85,3.75,649.5
CustomerID,388090.0,15297.050715,1710.309331,12347.0,13969.00,15159.00,16796.00,18287.0


In [27]:
df2 = df2[df2.UnitPrice!=0]

In [28]:
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,388057.0,278441.310302,152606.978516,0.00,148559.00,284834.00,409841.00,541908.0
InvoiceNo,388057.0,560616.136220,13092.004363,536365.00,549243.00,561889.00,572076.00,581587.0
Quantity,388057.0,12.667837,42.203930,1.00,2.00,6.00,12.00,4800.0
UnitPrice,388057.0,2.851500,3.983581,0.04,1.25,1.85,3.75,649.5
CustomerID,388057.0,15297.139536,1710.288514,12347.00,13969.00,15159.00,16796.00,18287.0


In [29]:
df2[df2.Quantity >1000]

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
3929,321304,565145,22492,MINI PAINT SET VINTAGE,1152,2011-09-01 13:50:00,0.55,12415,Australia
32329,19871,537899,22328,ROUND SNACK BOXES SET OF 4 FRUITS,1488,2010-12-09 10:44:00,2.55,12755,Japan
34093,511421,579498,23084,RABBIT NIGHT LIGHT,2040,2011-11-29 15:52:00,1.79,12798,Japan
37682,25920,538420,17096,ASSORTED LAQUERED INCENSE HOLDERS,1728,2010-12-12 12:03:00,0.17,12875,United Kingdom
38349,516562,579936,21787,RAIN PONCHO RETROSPOT,1200,2011-12-01 10:07:00,0.65,12901,United Kingdom
...,...,...,...,...,...,...,...,...,...
370296,424310,573261,22197,POPCORN HOLDER,1992,2011-10-28 12:32:00,0.72,17949,United Kingdom
376897,97432,544612,22053,EMPIRE DESIGN ROSETTE,3906,2011-02-22 10:43:00,0.82,18087,United Kingdom
377586,16436,537659,22188,BLACK HEART CARD HOLDER,1008,2010-12-07 16:43:00,2.31,18102,United Kingdom
377608,16435,537659,22189,CREAM HEART CARD HOLDER,1008,2010-12-07 16:43:00,2.31,18102,United Kingdom


In [30]:
df2[df2.UnitPrice >200]

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
44378,205759,554836,22655,VINTAGE RED KITCHEN CABINET,1,2011-05-26 16:25:00,295.0,13015,United Kingdom
54943,4989,536835,22655,VINTAGE RED KITCHEN CABINET,1,2010-12-02 18:06:00,295.0,13145,United Kingdom
71501,118769,546480,22656,VINTAGE BLUE KITCHEN CABINET,1,2011-03-14 11:38:00,295.0,13452,United Kingdom
71502,133994,547814,22656,VINTAGE BLUE KITCHEN CABINET,1,2011-03-25 14:19:00,295.0,13452,United Kingdom
179583,171178,551393,22656,VINTAGE BLUE KITCHEN CABINET,1,2011-04-28 12:22:00,295.0,14973,United Kingdom
190602,222682,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10 15:33:00,649.5,15098,United Kingdom


In [31]:
df2[(df2.Description.str.len())<15]['Description'].unique()

array(['DOORMAT HEARTS', 'RETRO MOD TRAY', 'PINK DOG BOWL',
       'SPOTTY BUNTING', 'RETROSPOT LAMP', 'OWL DOORSTOP',
       'LOCAL CAFE MUG', 'FIRST AID TIN', 'PARTY BUNTING',
       'JUMBO BAG OWLS', 'PHOTO CUBE', 'POPCORN HOLDER', 'WRAP COWBOYS  ',
       'WRAP CAROUSEL', 'SPACE FROG', 'SPACE OWL', 'MIRROR CORNICE',
       'POLKADOT PEN', 'POMPOM CURTAIN', 'LED TEA LIGHTS',
       'TOYBOX  WRAP ', 'POTTERING MUG', 'BINGO SET', 'KEY FOB , SHED',
       'BUTTON BOX ', 'FUNKY DIVA PEN', 'SANDALWOOD FAN', 'BLUE FLY SWAT',
       'CHILLI LIGHTS', 'GLAMOROUS  MUG', 'WICKER STAR ',
       'CUTE CATS TAPE', 'WRAP FOLK ART', 'SOMBRERO ', 'SKULLS TAPE',
       'DOGGY RUBBER', 'PACKING CHARGE', 'RIBBONS PURSE ',
       'PINK CAT BOWL', 'DAISY JOURNAL ', 'RAIN PONCHO ',
       'GARAGE KEY FOB', 'PINK FLY SWAT', 'BUNNY EGG BOX',
       'WRAP, CAROUSEL', 'BATHROOM HOOK', 'CHAMBRE HOOK', 'FROG CANDLE',
       'POSY CANDY BAG', 'GOLD WASHBAG', 'CORDIAL JUG'], dtype=object)

In [32]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388057 entries, 0 to 388089
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   388057 non-null  int64  
 1   InvoiceNo    388057 non-null  int64  
 2   StockCode    388057 non-null  object 
 3   Description  388057 non-null  object 
 4   Quantity     388057 non-null  int64  
 5   InvoiceDate  388057 non-null  object 
 6   UnitPrice    388057 non-null  float64
 7   CustomerID   388057 non-null  int64  
 8   Country      388057 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 29.6+ MB


Let's add a few features to the dataset.

In [34]:
df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])

In [35]:
# for the purpose of time series analysis, let's split the InvoiceDate into details.
df2['Day'] = df2['InvoiceDate'].dt.day
df2['Month'] = df2['InvoiceDate'].dt.month
df2['Year'] = df2['InvoiceDate'].dt.year
df2['Weekday'] = df2['InvoiceDate'].dt.day_name()
df2['Month_year'] = pd.to_datetime(df2[['Year', 'Month']].assign(Day=1))
df2['Day_Month_Year'] = pd.to_datetime(df2[['Year', 'Month', 'Day']])
df2['Hour'] = df2['InvoiceDate'].dt.hour

In [36]:
# adding a column of "revenue"
df2['Spending'] = df2['UnitPrice']*df2['Quantity']

In [37]:
# now check on how the data look like.
df2.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day,Month,Year,Weekday,Month_year,Day_Month_Year,Hour,Spending
0,148306,549222,16008,SMALL FOLDING SCISSOR(POINTED EDGE),24,2011-04-07 10:43:00,0.25,12347,Iceland,7,4,2011,Thursday,2011-04-01,2011-04-07,10,6.0
1,220583,556201,17021,NAMASTE SWAGAT INCENSE,36,2011-06-09 13:01:00,0.30,12347,Iceland,9,6,2011,Thursday,2011-06-01,2011-06-09,13,10.8
2,148289,549222,20665,RED RETROSPOT PURSE,6,2011-04-07 10:43:00,2.95,12347,Iceland,7,4,2011,Thursday,2011-04-01,2011-04-07,10,17.7
3,72249,542237,20719,WOODLAND CHARLOTTE BAG,10,2011-01-26 14:30:00,0.85,12347,Iceland,26,1,2011,Wednesday,2011-01-01,2011-01-26,14,8.5
4,286629,562032,20719,WOODLAND CHARLOTTE BAG,10,2011-08-02 08:48:00,0.85,12347,Iceland,2,8,2011,Tuesday,2011-08-01,2011-08-02,8,8.5


In [38]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388057 entries, 0 to 388089
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Unnamed: 0      388057 non-null  int64         
 1   InvoiceNo       388057 non-null  int64         
 2   StockCode       388057 non-null  object        
 3   Description     388057 non-null  object        
 4   Quantity        388057 non-null  int64         
 5   InvoiceDate     388057 non-null  datetime64[ns]
 6   UnitPrice       388057 non-null  float64       
 7   CustomerID      388057 non-null  int64         
 8   Country         388057 non-null  object        
 9   Day             388057 non-null  int64         
 10  Month           388057 non-null  int64         
 11  Year            388057 non-null  int64         
 12  Weekday         388057 non-null  object        
 13  Month_year      388057 non-null  datetime64[ns]
 14  Day_Month_Year  388057 non-null  dat

In [39]:
df2.to_csv(r'G:\My Drive\Jun\Mechine Learning\Data Science\Capstone Two\CleanData.csv')